In [1]:
import sys
sys.path.append("../") # go to parent dir




import iarray as ia
import numpy as np
from numcodecs import Blosc
import zarr
import dask
import dask.array as da
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import ctypes
mkl_rt = ctypes.CDLL('libmkl_rt.so')
mkl_set_num_threads = mkl_rt.MKL_Set_Num_Threads

mkl_set_num_threads(1)

-220680576

In [3]:
from ipython_memwatcher import MemWatcher

In [4]:
DTYPE = np.float64
NTHREADS = 8
CLEVEL = 5
CLIB = ia.LZ4
NREP = 1

In [5]:
shape = (16384, 16384)
shape = [int(x / 1) for x in shape]
chunkshape = (8192, 4096)
chunkshape = [int(x / 1) for x in chunkshape]
blockshape = (2048, 2048)
blockshape = [int(x / 1) for x in blockshape]


ashape = shape
achunkshape = chunkshape
ablockshape = blockshape

bshape = shape
bchunkshape = chunkshape
bblockshape = blockshape

cchunkshape = chunkshape
cblockshape = blockshape

daskchunks = chunkshape

In [6]:
cparams = dict(clib=CLIB, clevel=CLEVEL, nthreads=NTHREADS)

storage = ia.StorageProperties(achunkshape, ablockshape)

astorage = ia.StorageProperties(achunkshape, ablockshape)


lia = ia.linspace(ia.dtshape(ashape, dtype=DTYPE), 0, 1, storage=storage, **cparams)
nia = ia.random_normal(ia.dtshape(ashape, dtype=DTYPE), 0, 0.0000001, storage=storage, **cparams)
aia = (lia + nia).eval(storage=astorage, dtshape=ia.dtshape(ashape, dtype=DTYPE), **cparams)

bstorage = ia.StorageProperties(bchunkshape, bblockshape)

lia = ia.linspace(ia.dtshape(bshape, dtype=DTYPE), 0, 1, storage=storage, **cparams)
nia = ia.random_normal(ia.dtshape(bshape, dtype=DTYPE), 0, 0.0000001, storage=storage, **cparams)
bia = (lia + nia).eval(storage=bstorage, dtshape=ia.dtshape(bshape, dtype=DTYPE), **cparams)


cstorage = ia.StorageProperties(cchunkshape, cblockshape)

In [7]:
compressor = Blosc(cname='lz4', clevel=CLEVEL, shuffle=Blosc.SHUFFLE)


azarr = zarr.empty(shape=ashape,
                  chunks=daskchunks, dtype=DTYPE, compressor=compressor)
for info, block in aia.iter_read_block(daskchunks):
    sl = tuple([slice(i, i + s) for i, s in zip(info.elemindex, info.shape)])
    azarr[sl] = block[:]

bzarr = zarr.empty(shape=shape,
                  chunks=daskchunks, dtype=DTYPE, compressor=compressor)
for info, block in bia.iter_read_block(daskchunks):
    sl = tuple([slice(i, i + s) for i, s in zip(info.elemindex, info.shape)])
    bzarr[sl] = block[:]

In [8]:
mw = MemWatcher()
mw.start_watching_memory()

In [8] used 0.000 MiB RAM in 0.001s, peaked 0.000 MiB above current, total RAM usage 8124.316 MiB


In [9]:
scheduler = "single-threaded" if NTHREADS == 1 else "threads"
with dask.config.set(scheduler=scheduler, num_workers=NTHREADS):
    ad = da.from_zarr(azarr)
    bd = da.from_zarr(bzarr)
    cd = da.matmul(ad, bd)
    czarr = zarr.empty(shape=(ashape[0], bshape[1]),
                  chunks=daskchunks, dtype=DTYPE, compressor=compressor)
    da.to_zarr(cd, czarr)

In [9] used 1119.270 MiB RAM in 14.504s, peaked 17448.273 MiB above current, total RAM usage 9243.586 MiB


In [10]:
cia = ia.matmul(aia, bia, storage=cstorage, **cparams) #2GB

In [10] used 1042.312 MiB RAM in 17.311s, peaked 0.000 MiB above current, total RAM usage 10285.898 MiB


In [11]:
mw.stop_watching_memory()

In [12]:
np.testing.assert_allclose(czarr, ia.iarray2numpy(cia))